# API Data Wrangling Mini-Project

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case

import os

filename = os.path.join(os.path.expanduser('~'), 'OneDrive', 'Documents', 'quandl.key')
with open(filename, mode='r') as file:
    line = file.readline()
    while line:
        if line.startswith('key='):
            API_KEY = line.replace('key=', '')
            break
        line = file.readline()

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

https://www.quandl.com/data/FSE-Frankfurt-Stock-Exchange/documentation

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: https://requests.readthedocs.io/en/master/

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
# GET https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}
curl = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}?api_key={API_KEY}'

In [4]:
# Call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

req = requests.get(curl + '&limit=1')
req.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-09-08',
  'frequency': 'daily',
  'data': [['2020-09-08',
    95.8,
    96.0,
    92.95,
    95.0,
    None,
    74768.0,
    7063150.75,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [5]:
type(req.json())

dict

In [6]:
req.json().get('dataset_data').keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [7]:
req.json().get('dataset_data').get('data')

[['2020-09-08',
  95.8,
  96.0,
  92.95,
  95.0,
  None,
  74768.0,
  7063150.75,
  None,
  None,
  None]]

---

## Tasks for this mini project:

Using only the ***requests*** and Python 3 base modules

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Creat a class to perform the basic tasks for

* retrieving the time series
* return the high and low opening prices
* return the high and the low closing price
...

In [8]:
class FrankfurtStockExchange:
    '''
    a Python dictionary representation of the retrieved price, volume etc.
    of a stock listed on Frankfurt Stock Exchnage
    
    to-do: make data private
    to-do: set_date_range requests updated data
    '''

    def __init__(self, ticker, api_key, **kwargs):
        
        self.ticker = ticker
        self.api_key = api_key
        start_date=''; end_date=''
        
        for key, arg in kwargs.items():
            if key == 'start_date': start_date=arg
            if key == 'end_date': end_date=arg
                
        self.__request_data__(ticker, api_key, start_date, end_date)

        
    def __request_data__(self, ticker, api_key, start_date, end_date, limit=1):
        '''
        Request a data set via the API
        '''
        database_code = 'FSE'
        dataset_code = ticker
        return_format = 'json'
        
        if len(start_date) > 0 or len(end_date) > 0: limit=''

        # base method
        curl = f'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}_X/data.{return_format}?api_key={api_key}'

        req = requests.get(curl + f'&start_date={start_date}&end_date={end_date}&limit={limit}')
        
        # task 2: transform into a Python dict
        dataset = req.json()['dataset_data']
        columns = dataset['column_names']
        self.data = {column:[] for column in columns}
        for row in dataset['data']:
            for i, value in enumerate(row):
                self.data[columns[i]].append(value)
                
        # task 3
        # to-do: refactor to use sorted()
        self.max_open = max(list(filter(None, self.data['Open'])))
        self.min_open = min(list(filter(None, self.data['Open'])))
 
    
    # task 4
    def max_daily_price_diff(self):
        '''Returns the largest one-day change based on the daily high and low price'''
        # to-do: refactor this to take cond=min|max
        daily_change = 0
        for high, low in zip(self.data['High'], self.data['Low']):
            try:
                diff = high - low
                if diff > daily_change: daily_change = diff
            except:
                pass
        
        return daily_change

    # task 5
    def consective_close_diff(self, cond='max'):
        '''Returns a list of difference in the two consective closing price'''
        closing_prices = list(filter(None, self.data['Close'])) # drop missing values
        ## to-do: write dropna function where it takes list of value
        l = [closing_prices[i + 1] - closing_prices[i] for i in range(len(closing_prices) - 1)]
        
        return self.__abs_diff(l, cond)
    
    def __abs_diff(self, l, cond='max'):
        '''Given a list, find and return the max difference regardless of sign'''
        if cond == 'min':
            diff = float('inf')
        else:
            diff = 0
        
        for x in l:
            if cond == 'min':
                if abs(x) < abs(diff): diff = x            
            else:
                if abs(x) > abs(diff): diff = x
        
        return diff

    # task 6
    def mean(self, key):
        l = list(filter(None, self.data[key]))
        
        return sum(l) / len(l)

    # task 7
    def median(self, key):
        l = sorted(list(filter(None, self.data[key])))
        n = len(l)
        if n == 0: raise StatisticsError("empty list")
            
        if n%2 == 0:
            i = n//2
            return (l[i - 1] + l[i]) / 2
        else:
            return l[n//2]

        
    def get_date_range(self):
        return (min(self.data['Date']), max(self.data['Date']))
    
    def set_date_range(self, start_date, end_date):
        pass

## Start here

In [9]:
afx = FrankfurtStockExchange('AFX', API_KEY)
afx.get_date_range()

('2020-09-08', '2020-09-08')

1. Collect data

In [10]:
afx = FrankfurtStockExchange('AFX', API_KEY, start_date='2017-01-01', end_date='2017-12-31')

In [11]:
afx.get_date_range()

('2017-01-02', '2017-12-29')

2. Convert the received data into a Python dictionary

In [12]:
afx.data.keys()

dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])

In [13]:
afx.data['Turnover'][-10:]

[2065534.0,
 5703427.0,
 4369079.0,
 1528055.0,
 2157182.0,
 964046.0,
 1692326.0,
 1906810.0,
 2515473.0,
 1590561.0]

3. Highest and lowest opening price for this period

In [14]:
afx.max_open

53.11

In [15]:
afx.min_open

34.0

4. Largest daily change based on daily high and low prices

In [16]:
round(afx.max_daily_price_diff(), 2)

2.81

5. Largest change between to consecutive days by closing price

In [17]:
round(afx.consective_close_diff('max'), 2)

2.56

6. Average daily trading volume

In [18]:
round(afx.mean('Traded Volume'), 3)

89124.337

7. Median daily trading volume

In [19]:
round(afx.median('Traded Volume'), 2)

76286.0